In [ ]:
cd /content/drive/MyDrive/SeSac_도봉캠퍼스_AI_양수민/chapter4.Data_Handling

/content/drive/MyDrive/SeSac_도봉캠퍼스_AI_양수민/chapter4.Data_Handling


# Data Load

In [ ]:
import pandas as pd
import os

In [ ]:
file_path = './data'
file_name = 'book_data.csv'

df = pd.read_csv(os.path.join(file_path, file_name))

In [ ]:
df.head()

,ANALS_TRGET_YEAR,AREA_NM,AGE_VALUE,LON_CO,LON_MBER_CO
0,2019,강원도,영유아(0-5),1909.0,240
1,2019,강원도,유아(6-7),2437.0,282
2,2019,강원도,초등(8-13),13338.0,1606
3,2019,강원도,청소년(14-19),5019.0,1121
4,2019,강원도,20대,4040.0,904


# Data Basic Exploration
## Column 소개
- AREA_NM : 지역명
- LON_CO : 총 대출건수
- LON_MBER_CO : 대출해간 사람 수
- AGE_VALUE : 연령대
- ANALS_TRGET_YEAR : 대출년도

## Data 상위/하위 5개행 확인

In [ ]:
df.head()

,ANALS_TRGET_YEAR,AREA_NM,AGE_VALUE,LON_CO,LON_MBER_CO
0,2019,강원도,영유아(0-5),1909.0,240
1,2019,강원도,유아(6-7),2437.0,282
2,2019,강원도,초등(8-13),13338.0,1606
3,2019,강원도,청소년(14-19),5019.0,1121
4,2019,강원도,20대,4040.0,904


In [ ]:
df.tail()

,ANALS_TRGET_YEAR,AREA_NM,AGE_VALUE,LON_CO,LON_MBER_CO
121,2019,전라북도,20대,2616.000000,627
122,2019,전라북도,30대,5118.000000,705
123,2019,전라북도,40대,2224.761734,1240
124,2019,전라북도,50대,3160.000000,529
125,2019,전라북도,60대이상,3446.000000,486


## Data Shape확인

In [ ]:
df.shape

(126, 5)

## Column, Null data 확인

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANALS_TRGET_YEAR  126 non-null    int64  
 1   AREA_NM           126 non-null    object 
 2   AGE_VALUE         126 non-null    object 
 3   LON_CO            126 non-null    float64
 4   LON_MBER_CO       126 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.0+ KB


In [ ]:
import random
import numpy as np
for i in range(10):
  missing_value = random.randint(0, df.shape[0])
  df.loc[missing_value, 'LON_CO'] = np.nan

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANALS_TRGET_YEAR  126 non-null    int64  
 1   AREA_NM           126 non-null    object 
 2   AGE_VALUE         126 non-null    object 
 3   LON_CO            116 non-null    float64
 4   LON_MBER_CO       126 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.0+ KB


# 결측치 처리

In [ ]:
df.loc[df['LON_CO'].isnull(), ]

,ANALS_TRGET_YEAR,AREA_NM,AGE_VALUE,LON_CO,LON_MBER_CO
10,2019,경기도,유아(6-7),NaN,4812
31,2019,경상북도,20대,NaN,1281
32,2019,경상북도,30대,NaN,1832
35,2019,경상북도,60대이상,NaN,1058
39,2019,광주광역시,청소년(14-19),NaN,699
44,2019,광주광역시,60대이상,NaN,789
47,2019,대구광역시,초등(8-13),NaN,4380
55,2019,대전광역시,유아(6-7),NaN,499
87,2019,세종특별자치시,40대,NaN,1963
93,2019,울산광역시,청소년(14-19),NaN,1240


## 지역별 가중치 Data Frame 만들기

In [ ]:
df_area_grouped = df.loc[:, ["AREA_NM", "LON_CO"]].groupby("AREA_NM", as_index = False).mean()

In [ ]:
df_area_grouped['LON_CO'] = df_area_grouped['LON_CO'] / df_area_grouped['LON_CO'].max()

In [ ]:
df_area_grouped

,AREA_NM,LON_CO
0,강원도,0.047916
1,경기도,1.000000
2,경상남도,0.120119
3,경상북도,0.086139
4,광주광역시,0.046403
5,대구광역시,0.144194
6,대전광역시,0.092167
7,부산광역시,0.264275
8,서울특별시,0.615146
9,세종특별자치시,0.019898


## 결측치 채우기

In [ ]:
for idx, row in df.loc[df['LON_CO'].isnull(), ].iterrows():
  age = row['AGE_VALUE']
  area_nm = row['AREA_NM']
  mean_value = df.loc[df['AGE_VALUE'] == age, "LON_CO"].mean()
  weight = df_area_grouped.loc[df_area_grouped['AREA_NM'] == area_nm, 'LON_CO'].iloc[0]
  df.loc[idx, 'LON_CO'] = mean_value * weight

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANALS_TRGET_YEAR  126 non-null    int64  
 1   AREA_NM           126 non-null    object 
 2   AGE_VALUE         126 non-null    object 
 3   LON_CO            126 non-null    float64
 4   LON_MBER_CO       126 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.0+ KB


In [ ]:
for idx, row in df.loc[df['LON_CO'].isnull(), :].iterrows():
  age = row['AGE_VALUE']
  area_nm = row['AREA_NM']
  df.loc[idx, 'LON_CO'] = df.loc[df['AGE_VALUE'] == age, 'LON_CO'].mean() * df_area_grouped.loc[df_area_grouped['AREA_NM'] == area_nm, "LON_CO"].iloc[0]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 126 entries, 0 to 125
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ANALS_TRGET_YEAR  126 non-null    int64  
 1   AREA_NM           126 non-null    object 
 2   AGE_VALUE         126 non-null    object 
 3   LON_CO            126 non-null    float64
 4   LON_MBER_CO       126 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 5.0+ KB


## 파생변수 만들기
- 1인당 평균 독서량 칼럼(READ_QUANT) 만들기


In [ ]:
df['READ_QUANT'] = df['LON_CO'] / df['LON_MBER_CO']

# 통계적 기법을 활용하여 Data Analysis

## P-value

## T-Test 실습


In [ ]:
avg_read_quant_by_area = df.groupby("AREA_NM")['READ_QUANT'].mean()

In [ ]:
avg_read_quant_by_area

,READ_QUANT
AREA_NM,
강원도,6.137216
경기도,6.840565
경상남도,6.268210
경상북도,5.536472
광주광역시,5.813596
대구광역시,7.592824
대전광역시,6.733209
부산광역시,5.392972
서울특별시,6.156311


In [ ]:
import scipy.stats as stats

In [ ]:
t_test_results = pd.DataFrame(index = avg_read_quant_by_area.index, columns = avg_read_quant_by_area.index)

In [ ]:
group1 = df.loc[df["AREA_NM"] == "강원도", "READ_QUANT"]
group2 = df.loc[df['AREA_NM'] == "세종특별자치시", 'READ_QUANT']

In [ ]:
_, p_value = stats.ttest_ind(group1, group2, equal_var = True)

In [ ]:
p_value

0.7018327302871376

In [ ]:
for i, area1 in enumerate(avg_read_quant_by_area.index):
  for j, area2 in enumerate(avg_read_quant_by_area.index):
    if i == j:
      t_test_results.loc[area1, area2] = 1.0
    else:
      group1 = df.loc[df['AREA_NM'] == area1, 'READ_QUANT']
      group2 = df.loc[df['AREA_NM'] == area2, 'READ_QUANT']
      _, p_value = stats.ttest_ind(group1, group2, equal_var = True)
      t_test_results.loc[area1, area2] = p_value

In [ ]:
t_test_results

AREA_NM,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,세종특별자치시,울산광역시,인천광역시,전라남도,전라북도
AREA_NM,,,,,,,,,,,,,,
강원도,1.0,0.166765,0.911854,0.748509,0.556689,0.106118,0.157758,0.482448,0.986959,0.701833,0.568024,0.57731,0.427041,0.54932
경기도,0.166765,1.0,0.208413,0.277767,0.317482,0.726593,0.795757,0.051983,0.179587,0.084156,0.272666,0.32778,0.584261,0.452391
경상남도,0.911854,0.208413,1.0,0.839257,0.658007,0.132748,0.212319,0.433468,0.926817,0.629975,0.679079,0.674548,0.496703,0.627751
경상북도,0.748509,0.277767,0.839257,1.0,0.829539,0.176929,0.301222,0.320457,0.767178,0.483455,0.865113,0.840652,0.61657,0.764284
광주광역시,0.556689,0.317482,0.658007,0.829539,1.0,0.198311,0.344596,0.171186,0.582401,0.294996,0.942906,0.994212,0.719072,0.892084
대구광역시,0.106118,0.726593,0.132748,0.176929,0.198311,1.0,0.529017,0.035155,0.114495,0.055027,0.168498,0.206234,0.394875,0.29819
대전광역시,0.157758,0.795757,0.212319,0.301222,0.344596,0.529017,1.0,0.032324,0.175886,0.060691,0.276125,0.361803,0.702077,0.53313
부산광역시,0.482448,0.051983,0.433468,0.320457,0.171186,0.035155,0.032324,1.0,0.487324,0.719318,0.152784,0.193758,0.169493,0.229813
서울특별시,0.986959,0.179587,0.926817,0.767178,0.582401,0.114495,0.175886,0.487324,1.0,0.699175,0.596708,0.60124,0.444912,0.567928


## ANOVA(Analysis of Variance)

In [ ]:
anova_results = pd.DataFrame(index = avg_read_quant_by_area.index, columns = avg_read_quant_by_area.index)

for i, area1 in enumerate(avg_read_quant_by_area.index):
  for j, area2 in enumerate(avg_read_quant_by_area.index):

    group1 = df.loc[df['AREA_NM'] == area1, 'READ_QUANT']
    group2 = df.loc[df['AREA_NM'] == area2, 'READ_QUANT']
    _, p_value = stats.f_oneway(group1, group2)
    anova_results.loc[area1, area2] = p_value

In [ ]:
anova_results

AREA_NM,강원도,경기도,경상남도,경상북도,광주광역시,대구광역시,대전광역시,부산광역시,서울특별시,세종특별자치시,울산광역시,인천광역시,전라남도,전라북도
AREA_NM,,,,,,,,,,,,,,
강원도,1.0,0.166765,0.911854,0.748509,0.556689,0.106118,0.157758,0.482448,0.986959,0.701833,0.568024,0.57731,0.427041,0.54932
경기도,0.166765,NaN,0.208413,0.277767,0.317482,0.726593,0.795757,0.051983,0.179587,0.084156,0.272666,0.32778,0.584261,0.452391
경상남도,0.911854,0.208413,1.0,0.839257,0.658007,0.132748,0.212319,0.433468,0.926817,0.629975,0.679079,0.674548,0.496703,0.627751
경상북도,0.748509,0.277767,0.839257,1.0,0.829539,0.176929,0.301222,0.320457,0.767178,0.483455,0.865113,0.840652,0.61657,0.764284
광주광역시,0.556689,0.317482,0.658007,0.829539,1.0,0.198311,0.344596,0.171186,0.582401,0.294996,0.942906,0.994212,0.719072,0.892084
대구광역시,0.106118,0.726593,0.132748,0.176929,0.198311,1.0,0.529017,0.035155,0.114495,0.055027,0.168498,0.206234,0.394875,0.29819
대전광역시,0.157758,0.795757,0.212319,0.301222,0.344596,0.529017,1.0,0.032324,0.175886,0.060691,0.276125,0.361803,0.702077,0.53313
부산광역시,0.482448,0.051983,0.433468,0.320457,0.171186,0.035155,0.032324,1.0,0.487324,0.719318,0.152784,0.193758,0.169493,0.229813
서울특별시,0.986959,0.179587,0.926817,0.767178,0.582401,0.114495,0.175886,0.487324,1.0,0.699175,0.596708,0.60124,0.444912,0.567928
